In [ ]:
#Import all libraries
import numpy as np
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import string
# import seaborn as sns
# import plotly.graph_objects as go
from sklearn.decomposition import PCA
import warnings
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix , classification_report
from sklearn.preprocessing import LabelEncoder
import re
from collections import Counter
warnings.filterwarnings('ignore')

In [ ]:
#creating word cloud for given corpus  
def make_wordcloud(corpus , stopword):
    word_cloud = WordCloud(
            width=3000,
            height=2000,
            random_state=1,
            background_color="black",
            colormap="Pastel1",
            collocations=False,
            stopwords=stopword,
            ).generate(corpus)
    return word_cloud

In [ ]:
#This function swaps 2 columns inside the dataframe
def swap_columns(df, col1, col2):
    col_list = list(df.columns)
    x, y = col_list.index(col1), col_list.index(col2)
    col_list[y], col_list[x] = col_list[x], col_list[y]
    df = df[col_list]
    return df

In [ ]:
#This function removes punctuation from string
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [ ]:
#Reading dataset
Email_dataset = pd.read_csv("spam_ham_dataset.csv")

In [ ]:
# #Show dataset
# Email_dataset.head()

In [ ]:
s=Email_dataset["label"].value_counts()
s

In [ ]:
# create a bar chart
s.plot.bar()

# set the title and axis labels
plt.title("Email Label Counts")
plt.xlabel("Label")
plt.ylabel("Count")

# display the chart
plt.show()

In [ ]:
#Dropping columns that are not needed
Email_dataset = Email_dataset.drop('Unnamed: 0', axis=1)
Email_dataset = Email_dataset.drop('label', axis=1)

In [ ]:
# Email_dataset.head()

In [ ]:
#Creating a new feature, extracting subject of each email
subjects = []
for i in range(len(Email_dataset)):
    ln = Email_dataset["text"][i]
    line = ""
    for i in ln:
        if(i == '\r'):
            break
        line = line + i
    line = line.replace("Subject" , "")
    subjects.append(line)

In [ ]:
Email_dataset['Subject'] = subjects

In [ ]:
#Renaming the dataframe columns
Email_dataset.columns = ["Email_text" , "Labels" , "Email_Subject"]

In [ ]:
# Email_dataset.head()

In [ ]:
#Swapping the dataframe columns 
Email_dataset = swap_columns(Email_dataset, 'Labels', 'Email_Subject')

In [ ]:
# Email_dataset.head()

In [ ]:
#Converting all strings to lowercase
Email_dataset['Email_Subject'] = Email_dataset['Email_Subject'].str.lower()
Email_dataset['Email_text'] = Email_dataset['Email_text'].str.lower()

In [ ]:
#Removing Punctuation from the data
Email_dataset['Email_Subject'] = Email_dataset['Email_Subject'].apply(remove_punctuations)
Email_dataset['Email_text'] = Email_dataset['Email_text'].apply(remove_punctuations)

In [ ]:
# Email_dataset.head()

In [ ]:
#Creting seprate dataset for Spam and Non Spam emails, to perform analysis 
Spam = pd.DataFrame(columns = ['Email_text', 'Email_Subject', 'Labels'])
Non_Spam = pd.DataFrame(columns = ['Email_text', 'Email_Subject', 'Labels'])

In [ ]:
#Creating Non_Spam email dataset 
for i in range(len(Email_dataset)):
    if(Email_dataset['Labels'][i] == 0):
        new_row = {'Email_text':Email_dataset['Email_text'][i], 'Email_Subject':Email_dataset['Email_Subject'][i], 'Labels':Email_dataset['Labels'][i]}
        Non_Spam.loc[len(Non_Spam)] = new_row

In [ ]:
#Creating Spam email dataset 
for i in range(len(Email_dataset)):
    if(Email_dataset['Labels'][i] == 1):
        new_row = {'Email_text':Email_dataset['Email_text'][i], 'Email_Subject':Email_dataset['Email_Subject'][i], 'Labels':Email_dataset['Labels'][i]}
#         Spam = Spam.append(new_row, ignore_index=True)
        Spam.loc[len(Spam)] = new_row

In [ ]:
# Spam.head()

In [ ]:
# Non_Spam.head()

In [ ]:
# #create a corpus conatininga all the email text 
# Email_text_corpus = []
# for line in Email_dataset['Email_text']:
#     words = line.split(" ")
#     Email_text_corpus.append(words)

In [ ]:
# #create a word2vec model for words in email text 
# model = Word2Vec(Email_text_corpus, min_count=1, vector_size=56)

In [ ]:
# X = model.wv[model.wv.key_to_index ]
# pca = PCA(n_components=2)
# result = pca.fit_transform(X)
# pca_df = pd.DataFrame(result, columns = ['x','y'])

In [ ]:
# #visualizing the corpus as a word vector
# N = 1000000
# words = list(model.wv.key_to_index)
# fig = go.Figure(data=go.Scattergl(
#    x = pca_df['x'],
#    y = pca_df['y'],
#    mode='markers',
#    marker=dict(
#        color=np.random.randn(N),
#        colorscale='Viridis',
#        line_width=1
#    ),
#    text=words,
#    textposition="bottom center"
# ))

# fig.show()

In [ ]:
# Email_dataset.head()

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
def cost_function(h, y):
    return (-y * np.log(h + 1e-10) - (1 - y) * np.log(1 - h + 1e-10)).mean()

Gradient Descent is used while training a machine learning model. It's an optimization algorithm, used to minimize some function by iteratively moving in the direction of steepest descent as defined by the negative of the gradient.

In [ ]:
def gradient_descent(X, h, y):
    return np.dot(X.T, (h - y)) / y.shape[0]

Weights are updated iteratively by taking steps proportional to the negative of the gradient.

In [ ]:
def update_weight_loss(weight, learning_rate, gradient):
    return weight - learning_rate * gradient

Convert the 'Email_text' column to a matrix of token counts.

In [ ]:
X = Email_dataset['Email_text']
y = Email_dataset['Labels'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Fit the CountVectorizer on your training data
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train).toarray()

# Transform your test data using the same vectorizer
X_test = vectorizer.transform(X_test).toarray()

In [ ]:
num_iter = 1000
learning_rate = 0.1
weight = np.zeros(X_train.shape[1])

In [ ]:
costs = []

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
ConfusionMatrixDisplay.from_estimator(classifier, X_test, y_test)


# creating decision_tree_classification

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(random_state = 0)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

In [45]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)
accuracy_score(y_test, y_pred)


[[705  37]
 [ 18 275]]


0.9468599033816425

# Modified Naïve Bayes

In [46]:
total_emails = len(Email_dataset)

#prior probabilities
spam_not_spam_counter = Counter(Email_dataset["Labels"])
prior_not_spam = spam_not_spam_counter[0] / total_emails
prior_spam = spam_not_spam_counter[1] / total_emails
print(prior_not_spam)
print(prior_spam)

0.7101140978534133
0.2898859021465867


In [47]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
for word in stop_words:
    word = remove_punctuations(word)
# stop_words = {word.translate(str.maketrans('', '', string.punctuation)) for word in stop_words}
custom_stop_words = {
    "gave",
    "see",
    "etc"
}

stop_words.update(custom_stop_words)
# print(stop_words)

# def remove_words(text, words):
#     for word in words:
#         text = text.replace(word, '')
#     return text

# def remove_nums(text):
#     text = ''.join([i for i in text if not i.isdigit()])
#     return text

# non_spam_body = Non_Spam["Email_text"].apply(lambda x: remove_words(x, stop_words)).apply(lambda x: remove_nums(x))
# spam_body = Spam["Email_text"].apply(lambda x: remove_words(x, stop_words)).apply(lambda x: remove_nums(x))

def remove_stopwords(body):
    return ' '.join([word for word in body.split() if word not in stop_words and not word.isdigit() and not re.match(r'(\w)\1{1,}', word) and not len(word) <= 2])

def remove_single_words(set, set2):
    text = ' '.join(set2)
    text = text.split()
    word_counts = Counter(text)
    single_occurrence_words = {text for text, count in word_counts.items() if count <= 35}
    return [' '.join(filter(lambda text: text not in single_occurrence_words, set.split())) for set in set]

def count_and_combine(set):
    return dict(Counter("".join(set).split()))


def smoothing(set1, set2, smoothing:int, reg:int):
    first_set = set("".join(set1).split())
    second_set = set("".join(set2).split())
    first_dict = count_and_combine(set1)
    second_dict = count_and_combine(set2)
    for word in first_dict:
        first_dict[word] += smoothing
    
    for word in second_dict:
        second_dict[word] += smoothing
    
    if first_set != second_set:
        for word in first_set:
            if word not in second_set:
                second_dict[word] = 1
        
        for word in second_set:
            if word not in first_set:
                first_dict[word] = 1
    
    temp_first_dict = first_dict.copy()
    for word in temp_first_dict:
        if first_dict[word] < reg and second_dict[word] < reg:
            del first_dict[word]
            del second_dict[word]

    temp_second_dict = second_dict.copy()
    for word in temp_second_dict:
        if second_dict[word] < reg and first_dict[word] < reg:
            del first_dict[word]
            del second_dict[word]
    return [first_dict, second_dict]

def calc_prob(set):
    set_len = len(set)
    set_total = 0

    for word in set:
        set_total += set[word]
    
    prob = set
    for word in set:
        prob[word] = set[word] / set_total
    
    return prob


# Remove stopwords from each paragraph in the list
first_non_spam_body = [remove_stopwords(body) for body in Non_Spam["Email_text"]]
first_spam_body = [remove_stopwords(body) for body in Spam["Email_text"]]
# print(first_non_spam_body)

smoothing_list = smoothing(first_non_spam_body, first_spam_body, 1, 20)
first_non_spam_body = smoothing_list[0]
first_spam_body = smoothing_list[1]

non_spam_prob = calc_prob(first_non_spam_body)
spam_prob = calc_prob(first_spam_body)
for word in spam_prob:
    print('"' + word + '": ' + str(spam_prob[word]) + ',')

# print(first_non_spam_body)
# print(non_spam_prob_len)
# print(first_spam_body)
# print(spam_prob_len)

# total_spam_body = non_spam_body + spam_body
# non_spam_body = remove_single_words(non_spam_body, total_spam_body)
# spam_body = remove_single_words(spam_body, total_spam_body)




# print(Spam["Email_text"])
# print(set(spam_body))


# vectorizer = CountVectorizer()
# #vectorizer.fit(X_train)
# non_spam_words = vectorizer.fit_transform(non_spam_body)
# non_spam_word_list = vectorizer.get_feature_names_out()
# non_spam_word_count_list = [0 for e in non_spam_word_list]
# non_spam_count_total = 0

# for line in non_spam_words.toarray():
#     i = 0
#     for count in line:
#         non_spam_word_count_list[i] += count
#         non_spam_count_total += count
#         i += 1
# non_spam_word_count_dict = dict(zip(non_spam_word_list, non_spam_word_count_list))

# # temp_non_spam_word_count_list = non_spam_word_count_list
# # for word in temp_non_spam_word_count_list:
# #     if non_spam_word_count_list[word] == 1:
# #         del non_spam_word_count_list[word]
# non_spam_prob = non_spam_word_count_dict
# print(non_spam_prob)

# for word in non_spam_word_list:
#     non_spam_prob[word] = non_spam_word_count_dict[word] / non_spam_count_total
# print(non_spam_prob)


# vectorizer2 = CountVectorizer()
# #vectorizer.fit(X_train)
# spam_words = vectorizer2.fit_transform(spam_body)
# spam_word_list = vectorizer2.get_feature_names_out()
# # print(set(spam_word_list))
# spam_word_count_list = [0 for e in spam_word_list]
# spam_count_total = 0

# for line in spam_words.toarray():
#     i = 0
#     for count in line:
#         spam_word_count_list[i] += count
#         spam_count_total += count
#         i += 1
# spam_word_count_dict = dict(zip(spam_word_list, spam_word_count_list))

# # temp_spam_word_count_list = spam_word_count_list
# # for word in temp_spam_word_count_list:
# #     if spam_word_count_dict[word] == 1:
# #         del spam_word_count_dict[word]
# spam_prob = spam_word_count_dict
# print(spam_prob)

# for word in spam_word_list:
#     spam_prob[word] = spam_word_count_dict[word] / spam_count_total
# print(spam_prob)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\jandahuang/nltk_data'
    - 'c:\\Users\\jandahuang\\AppData\\Local\\anaconda3\\nltk_data'
    - 'c:\\Users\\jandahuang\\AppData\\Local\\anaconda3\\share\\nltk_data'
    - 'c:\\Users\\jandahuang\\AppData\\Local\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\jandahuang\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [ ]:
# spam_example = """Subject: Urgent Opportunity for Massive Profits!
# Dear [Recipient],
# Are you tired of struggling to make ends meet? Are you ready to unlock the secret to financial freedom? Look no further! Our revolutionary system guarantees massive profits with minimal effort.
# With our proven method, you can start earning thousands of dollars from the comfort of your own home. No experience required! Simply follow our step-by-step guide, and watch your bank account grow.
# But act fast! This exclusive opportunity won't last long. Join the ranks of our satisfied customers who are already living the life they've always dreamed of.
# Click the link below to get started today!
# [Link to dubious website]
# Don't miss out on this life-changing opportunity!
# Best regards,
# [Spam Sender]
# P.S. Still not convinced? Check out these testimonials from our happy customers!"""
# spam_example = """Summer Classes Still Available
# Summer Session 2024 starts soon, but there’s still time to register! Enroll early to guarantee your seat in the class.

# With 800+ class sections to choose from, Summer Session 2024 is a great way to:

# Raise your GPA.
# Take the classes you couldn’t get into in fall or spring.
# Reach the 30 units per year you need to graduate on time.
# With the majority of classes being online or hybrid, Summer Session will add flexibility to your schedule.

# To register for Summer Session 2024:
# Click the button below to go to your Student Portal.
# Click Student Homepage.
# Click Manage Classes tile.
# Click on Class Search and Enroll.
# Add classes to your cart the same way you would for fall or spring (make sure you set the Course Career to Extended Education).
# Register Now
# Note: Disney Aspire does not cover summer tuition.

# If you have any questions, please contact the TITAN Helpline at (657) 278-7601 or email the CSUF Registration Office at arsc@fullerton.edu.
# facebook	 	Instagram	 	twitter	 	 	Linkedin	 	Forward to Friend
# You received this email because you signed up to receive updates
# from CSUF Extension & International Programs. We respect your privacy.
# You can update your preferences or unsubscribe.
# CSUF Extension & International Programs
# 1111 N. State College Blvd., Fullerton, CA, 92831
# Need to mail us something? Please use the following address when sending postal mail:
# 800 N. State College Blvd., #THall 1403, Fullerton, CA, 92831

# © California State University, Fullerton, All rights reserved.
# """

# spam_example = """Identity Verification Complete
# Your identity verification was successful.

# Log In
# Why did this happen?
# Getting started with Netlify is quick, but this also means that bad actors try to abuse our service to host illegal content, phishing sites, spam, or similar content against our terms of service.

# To protect against this and help make the web safe for everyone, our automated fraud detection system will compare any new sign up against the patterns of fraudulent users, and if there is a high enough correlation, we will trigger this identity verification process.

# Verifying your identity only takes a moment and allows you access to Netlify’s services.

# Thank you for your understanding and cooperation."""
spam_example = """
subject: [CPSC 490.01] Important Notice: No Class and Final Proposal Due on upcoming Tuesday (May 7)

CSUF external service. Use caution and confirm sender.

Dear everyone,

I would like to inform you that there will be no class on the upcoming Tuesday, May 7. Instead, your only remaining task for this course is to submit your group Final Proposal by midnight on May 7 via Canvas.

Additionally, please note that there will be no final exam for this course. You can focus your efforts on preparing for your other classes during the final exam week.

Thank you for your dedication throughout the semester. If you have any questions or need further clarification, please do not hesitate to reach out.


Best regards,

Rong Jin"""
spam_example = spam_example.lower()
spam_example = re.sub(r'[^\w\s]', '', spam_example).split()

total_spam_prob = prior_spam
for word in spam_example:
    if word in spam_prob:
        print(word + " " + str(total_spam_prob) + " * " + str(spam_prob[word]) + " = " + str(total_spam_prob * spam_prob[word]))
        total_spam_prob = total_spam_prob * spam_prob[word]


total_non_spam_prob = prior_not_spam
print(total_non_spam_prob)
for word in spam_example:
    if word in non_spam_prob:
        print(word + " " + str(total_non_spam_prob) + " * " + str(non_spam_prob[word]) + " = " + str(total_non_spam_prob * non_spam_prob[word]))
        total_non_spam_prob = total_non_spam_prob * non_spam_prob[word]


print(total_spam_prob)
print(total_non_spam_prob)

if total_spam_prob >= total_non_spam_prob:
    print("Spam")
else:
    print("Not Spam")

In [ ]:
# from sklearn.naive_bayes import BernoulliNB
# from sklearn.metrics import confusion_matrix, accuracy_score

# bnb = BernoulliNB()

# bnb.fit(X_train, y_train)
# y_pred_class = bnb.predict(X_test)
# y_pred_probability = bnb.predict_proba(X_test)
# # metrics.accurarcy_socre(y_test, y_pred_class)

# cm = confusion_matrix(y_test,y_pred_class)
# print(cm)
# print(accuracy_score(y_test, y_pred_class))

# training Multinomial Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.pipeline import make_pipeline

cv = CountVectorizer()
nb = MultinomialNB()
nb.fit(X_train, y_train)

y_pred_class = nb.predict(X_test)
print(accuracy_score(y_test, y_pred_class))


0.9719806763285024


### Making Support Vector

In [ ]:
print(X_test)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 3 0 ... 0 0 0]]


In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel='rbf', random_state=42)
classifier.fit(X_train, y_train)


SVC(random_state=42)

# KNN


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# Vectorize the text data using the Bag-of-Words model
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

NameError: name 'X_train' is not defined

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
k = 5
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [ ]:
y_pred = knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)


Accuracy: 0.8618357487922705


# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFR

cl_rf = RFR(n_estimators=100, random_state=42)
cl_rf.fit(X_train, y_train)
# y_pred = cl_rf.predict(X_test)

RandomForestClassifier(random_state=42)

In [ ]:
y_pred = cl_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.9719806763285024


# Support Vector

In [ ]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train, y_train)

SVR()

In [ ]:
import math
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_percentage_error

maxY=max(y_test)
minY=min(y_test)
a=maxY-minY
print ("The R-Squared value is: "+str(r2_score(y_test, y_pred)))
print ("The Mean Squared Error value is: "+str(mean_squared_error(y_test, y_pred)))
print ("The Root Mean Squared Error value is: "+str(math.sqrt(mean_squared_error(y_test, y_pred))))
print ("The Normalize Root Mean Squared Error value is: "+str(math.sqrt(mean_squared_error(y_test, y_pred))/a*100))
print ("The Mean Absolute Error value is: "+str(mean_absolute_percentage_error(y_test, y_pred)*100))

The R-Squared value is: 0.8619403328335005
The Mean Squared Error value is: 0.028019323671497585
The Root Mean Squared Error value is: 0.16738973586064823
The Normalize Root Mean Squared Error value is: 16.738973586064823
The Mean Absolute Error value is: 5221564785357097.0


In [ ]:
y_pred = classifier.predict(X_test)

cm = confusion_matrix(y_test, y_pred)
print(cm)
print(accuracy_score(y_test, y_pred))

[[720  22]
 [ 14 279]]
0.9652173913043478
